# 🔐 Encrypto-Anonymizer

**A simple Google Colab-based tool by Elizabeth Shamblin to anonymize and encrypt CSV data securely.**

## 🔒 About the Encrypto-Anonymizer
- Uploaded files are stored **temporarily** on the virtual machine running your Colab session.
- Only **you** (the logged-in Google account) have access to them during that session.
- Files are **not shared** with Google, the public, or other users unless you explicitly do so.
- Colab does **not store** those files persistently — they are deleted when the session ends (e.g., if idle for 90 minutes or closed).


## 📁 Step 1: Upload a CSV file to anonymize

In [ ]:
from google.colab import files
import pandas as pd

uploaded = files.upload()
for fn in uploaded:
    filename = fn
    df = pd.read_csv(fn)
df.head()

## 🧩 Step 2: Select columns to anonymize

In [ ]:
print("Columns in your file:", list(df.columns))
columns_to_anonymize = ['Name', 'Email', 'ID']  # Change as needed

## 🔀 Step 3: Anonymize selected columns and create mapping

In [ ]:
import hashlib, uuid
from cryptography.fernet import Fernet
import json

# Generate encryption key
fernet_key = Fernet.generate_key()
fernet = Fernet(fernet_key)

mapping = {}

anon_df = df.copy()
for col in columns_to_anonymize:
    anon_ids = []
    for original_value in df[col]:
        anon_id = str(uuid.uuid4())
        encrypted_value = fernet.encrypt(str(original_value).encode()).decode()
        mapping[anon_id] = encrypted_value
        anon_ids.append(anon_id)
    anon_df[col] = anon_ids

anon_df.to_csv("anonymized_output.csv", index=False)
with open("encrypted_mapping.json", "w") as f:
    json.dump(mapping, f)
with open("encryption_key.key", "wb") as f:
    f.write(fernet_key)
print("✅ Anonymization complete.")

## 📥 Step 4: Download anonymized data and key

In [ ]:
files.download("anonymized_output.csv")
files.download("encrypted_mapping.json")
files.download("encryption_key.key")